In [1]:
import numpy as np

# Input data and parameters
arr = np.array([[10.0, 15.0], [3.0, 3.5],  [1.8, 3.0], [8.0, 10.0], [1.0,  2.0], [16.3, 17.0]])
duration = 4
length = 24.31
N = 20
step = 0.1

# Sort intervals by start time (0th column)
arr = arr[arr[:, 0].argsort()]

# Merge overlapping intervals
i = 0
while i < len(arr) - 1:
    if arr[i + 1, 0] <= arr[i, 1]:  # If next interval starts before current ends, merge
        arr[i, 1] = max(arr[i, 1], arr[i + 1, 1])
        arr = np.delete(arr, i + 1, 0)
    else:
        i += 1

# Calculate windows between intervals
start = 0
windows = []
for i in range(len(arr)):
    windows.append([arr[i, 0] - start, start, arr[i, 0]])
    start = arr[i, 1]
windows.append([length - arr[-1, 1], arr[-1, 1], length])  # Final window

windows = np.array(windows)
windows = windows[windows[:, 0].argsort()]  # Sort by window size

# Function to truncate floats
def truncate_float(float_number, decimal_places):
    multiplier = 10 ** decimal_places
    return int(float_number * multiplier) / multiplier


# Calculate the maximum possible clips and proportional distribution
first = windows[1,0]
MaxPossible = np.array([])
total_clips = 0 
for win in windows:
    if win[0] >= duration:
        max_clips = (win[0] - duration) / step + 1
        total_clips += max_clips
        MaxPossible.append(max_clips)
    else:
        MaxPossible.append(0)
        #windows.delete(row)

# Calculate proportional clips for each window
proportional = [N * max_clip / total_clips for max_clip in MaxPossible]

# Generate valid clips based on proportional distribution
valid_clips = []
remaining = N

for idx, window in enumerate(windows): 
    if MaxPossible[idx] == 0:
        continue

    w = int(proportional[idx] + 1)
    w = min(w, remaining)
    remaining -= w

    new_step = (window[0] - duration) / (w - 1) if w > 1 else 0
    start = window[1]

    for j in range(w):
        valid_clips.append(truncate_float(start, 3))
        if j < w - 1:
            start += new_step
        else:
            start = window[2] - duration

# Validate and print results
assert len(valid_clips) == N
print('valid_clips', len(valid_clips), valid_clips)


valid_clips 20 [3.5, 3.75, 4.0, 17.0, 17.206, 17.413, 17.62, 17.827, 18.034, 18.241, 18.448, 18.655, 18.861, 19.068, 19.275, 19.482, 19.689, 19.896, 20.103, 20.31]
